In [1]:
import yfinance as yf
import pandas as pd
import numpy as np


In [6]:
indices = ['^GSPC', '^IXIC', '^DJI', '^FTSE', '^N225']
equities = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'JNJ']


In [7]:
start_date = '2010-01-01'

data = yf.download(indices + equities, start=start_date)['Adj Close']


[*********************100%***********************]  10 of 10 completed


In [9]:
returns = data.pct_change().dropna()

In [11]:
cumulative_returns = (1 + returns).cumprod()


In [12]:
rolling_max = cumulative_returns.expanding().max()
drawdown = cumulative_returns / rolling_max - 1
max_drawdown = drawdown.min()


In [13]:
risk_free_rate = 0.0 
daily_risk = returns.std()
sharpe_ratio = (returns.mean() - risk_free_rate) / daily_risk


In [14]:
downside_returns = returns[returns < 0]
downside_risk = downside_returns.std()
sortino_ratio = (returns.mean() - risk_free_rate) / downside_risk


In [23]:
volatility = returns.std(axis=0)

In [27]:
results = pd.DataFrame({
    'Max Drawdown': max_drawdown,
    'Sharpe Ratio': sharpe_ratio,
    'Sortino Ratio': sortino_ratio
})


In [16]:
results.to_csv('assignment_results.csv')
